## Text Splitters : 문서를 다양하게 자르는 기능  
    - LLM API 불러오고 프롬프트를 탬플릿으로 설정하고, 다양한 문서를 불러오는 방법을 공부했다.  
    - RAG 시스템 상에서 문서를 불러온 후, 벡터 DB에 저장해야 한다.  
      - 사용자 질문 -> Document Loader - 임베딩 모델 -> 벡터 DB (사용자 질문, 유사 문장) -> LLM  
      - 도식화 하면 위와 같다.   
    - 문서를 Document Loader로 불러오면 이를 임베딩 형태로 변환하여 벡터 DB로 저장할 수 있다. 그러면 사용자 질문과 비교하여 유사 문장을 검색할 수 있는 준비 완료  
    - 그러나 문서가 긴 경우 한꺼번에 문서를 벡터 DB로 변환하는 것은 좋지 않다.   
      1. 임베딩 모델의 컨텍스트 윈도우 문제 
        : LLM처럼 임베딩 모델도 길이 제한이 있다. max_seq_length보다 길면 벗어난 만큼의 텍스트는 잘라내고 범위 안만 임베딩 변환 수행한다.     
      2. LLM의 컨텍스트 윈도우 문제   
        : 마찬가지로 LLM도 입력과 출력에서 길이 제한이 있다. 입력인 사용자 질문과 벡터 DB 내 유사 문서를 합친 총 텍스트의 길이가 제한을 넘으면 오류가 발생한다.   
      3. 건초더미에서 바늘 찾기 문제   
        : 길이가 긴 입력을 LLM에게 주면 컨텍스트의 정보를 처리하는 과정에서 앞부분을 다소 망각하는 현상이 벌어진다. 따라서 최대한 사용자 질문과 유사한 텍스트를 짧고 알차게 LLM에게 전달해야 한다.   
    - 적당한 크기로 문서를 분할하는 Text Splitters  
      - CharacterTextSplitter, RecursiveCharacter, SemanticChunker   
      - Document를 특정 기준에 따라 정해진 길이의 청크(chunk)로 분할한다. 따라서 하나의 문서가 사용자 정의 길이보다 길면, 이를 분할하여 여러개의 청크로 저장하는 것.   
      - 그러면 사용자 질문과 벡터 DB내 모든 청크 간의 유사도를 계산하여 유사도가 높은 N개의 청크를 검색할 수 있다.   
      - 이렇게 검색된 청크가 사용자의 질문과 합쳐져 LLM에게 전달할 최종 프롬프트가 된다.       

In [3]:
# 글자 수로 분할하는 CharacterTextSplitter
# 주요 매개변수 : separator(주어진 구분자를 기준으로 텍스트 분할하며 청크 사이즈에 맞게 재구성, "\n"), chunk_size(청크 사이즈), chunk_overlap(청크간 중첩 사이즈)
# length_function(청크 사이즈를 계산하는 함수, 기본은 len으로 글자 수 기준으로 청크 사이즈 정함), is_separator_regex(정규표현식으로 해석할지 여부 결정, boolean)

import warnings 
warnings.filterwarnings("ignore")

# PyPDFium2Loader로 PDF 문서 로드
from langchain.document_loaders import PyPDFium2Loader

loader = PyPDFium2Loader("./data/[이슈리포트 2022-2호] 혁신성장 정책금융 동향.pdf")
pages = loader.load()

# CharacterTextSplitter 모듈 불러오기 
from langchain_text_splitters import CharacterTextSplitter 

# 구분자: 줄 바꿈, 청크 길이: 500, 청크 오버랩: 100, length_function: 글자 수
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=500,
    chunk_overlap=100,
    length_function=len
) 

# 텍스트 분할
text = text_splitter.split_documents(pages)
print(text[0])

page_content='혁신성장 정책금융 동향 : ICT 산업을 중심으로
 CIS이슈리포트 2022-2호 | 1 |
<요 약>
▶ 혁신성장 정책금융기관*은 혁신성장산업 영위기업을 발굴·지원하기 위한 정책금융 가이드라인**에 따라 혁신
성장 기술분야에 대한 금융지원을 강화하고 있음
 * 산업은행, 기업은행, 수출입은행, 신용보증기금, 기술보증기금, 중소벤처기업진흥공단, 무역보험공사 등 
11개 기관
 ** 혁신성장 정책금융 지원 대상을 판단하는 기준으로, ‘9대 테마 – 46개 분야 – 296개 품목’으로 구성
￮ 정책금융기관의 혁신성장 정책금융 공급규모는 2017년 24.1조 원에서 2021년 85.4조 원으로 크게 증가
하여 국내 산업 구조의 미래 산업으로의 전환을 충실히 지원하고 있음
￮ 본 보고서는 ICT 산업의 정책금융 지원 트렌드를 파악하고, 혁신성장 정책금융이 집중되는 주요 품목의
기술·시장 동향을 분석함' metadata={'source': './data/[이슈리포트 2022-2호] 혁신성장 정책금융 동향.pdf', 'page': 0}


실행결과는 첫번째 페이지의 500자까지만 줄 바꿈을 기준으로 텍스트 분할된걸 알 수 있다.  

In [4]:
print(text[1])

page_content='￮ 본 보고서는 ICT 산업의 정책금융 지원 트렌드를 파악하고, 혁신성장 정책금융이 집중되는 주요 품목의
기술·시장 동향을 분석함
▶ 혁신성장 ICT 산업은 정보통신(6개 분야, 47개 품목), 전기전자(5개 분야, 27개 품목), 센서측정(3개 분야, 19개 품목) 테마로 구성되며, 혁신성장 정책금융기관의 공급액 규모는 2021년 말 기준 16.9조 원으로 
2017년 이후 연평균 39.2% 지속 증가하고 있음
￮ ICT 산업의 공급액 규모 비중은 혁신성장 정책금융 총 공급 규모의 약 20% 수준임
 * (‘17)18.7% → (’18)20.7% → (’19)18.5% → (’20)20.3% → (’21)19.8%
혁신성장 정책금융 동향 :
ICT 산업을 중심으로
오동찬 선임조사역 (dcoh@kcredit.or.kr)' metadata={'source': './data/[이슈리포트 2022-2호] 혁신성장 정책금융 동향.pdf', 'page': 0}


두번째 청크 출력 결과는 첫번째 페이지의 마지막 문장부터 500자 내의 텍스트다. 이는 chunk_overlap을 100으로 설정하여 텍스트 분할 시 앞 청크의 마지막 100자를 뒷 청크의 첫 100자로 지정했기 때문이다.   
이처럼 청크 간 문맥을 일부 공유하게끔 하여 LLM이 여러 청크를 참고할 때 올바르게 문맥을 참고하여 답변 생성할 수 있게 된다.

그러나 구분자 매개변수가 1개이기 때문에(separator), 청크 길이가 넘을 수도 있다.   
만약 청크길이가 500자로 설정됐고, 구분을 "\n" 즉 줄바꿈으로 하는데 만일 줄 바꿈을 하지 않고 한 문장이 500자가 넘는 텍스트가 있으면 이것을 하나의 청크로 구성하게 된다.   
실제 CharacterTextSplitter로 텍스트를 분할한 후 각 청크 길이를 세어보면 설정한 청크 사이즈가 아닌채로 구성한 텍스트가 존재할 수 있다.  

In [6]:
from langchain.document_loaders import PyPDFium2Loader

loader = PyPDFium2Loader("./data/[이슈리포트 2022-2호] 혁신성장 정책금융 동향.pdf")
pages = loader.load()

from langchain_text_splitters import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    separator="\n\n", # 문단 나누기
    chunk_size=500,
    chunk_overlap=100,
    length_function=len,
    is_separator_regex=False
) 

texts = text_splitter.split_documents(pages)
print([len(i.page_content) for i in texts])

[802, 1424, 1181, 1228, 708, 669, 1580, 567, 1412, 1302, 1106, 543, 1291, 1116, 1321, 945, 981, 1057]


구분자를 문단 나누기로 설정하면 각 문단 하나하나가 청크로 지정되는데 chunk_size 설정인 500을 넘겨서 1000이 넘는 청크들로 분할되는게 확인된다.   
사용자가 LLM의 컨텍스트 윈도우를 고려해서 chunk_size를 엄격하게 관리해야 한다면 이 부분은 단점이다.   
이럴 때를 고려하여 RecursiveCharacterTextSplitter 모듈을 사용한다. 

In [8]:
# 재귀적인 텍스트 분할 방법 RecursiveCharacterTextSplitter 
# 구분자가 하나가 아니라 여러개를 재귀적으로 적용하여 텍스트를 분할한다. (재귀적이라는건 문제가 해결될때까지 구분자 하나를 계속 호출하고 그 다음 구분자를 실행하는 것)
# 매개변수 : separator(구분자 리스트: 지정된 청크 길이를 지킬 수 있도록 리스트를 재귀적으로 적용 ("\n\n", "\n", " ", ""))
# 나머지 매개변수들은 CharacterTextSplitter와 동일

from langchain.document_loaders import PyPDFium2Loader

loader = PyPDFium2Loader("./data/[이슈리포트 2022-2호] 혁신성장 정책금융 동향.pdf")
pages = loader.load()

from langchain_text_splitters import RecursiveCharacterTextSplitter

text_spliter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", " ", ""], 
    chunk_size=500,
    chunk_overlap=100,
    length_function=len,
    is_separator_regex=False
)

texts = text_spliter.split_documents(pages)
print([len(i.page_content) for i in texts])

[460, 415, 443, 492, 464, 304, 477, 493, 340, 484, 464, 436, 464, 312, 482, 235, 493, 467, 464, 353, 493, 152, 494, 467, 487, 198, 487, 484, 463, 449, 489, 285, 489, 108, 454, 440, 459, 136, 473, 494, 299, 497, 492, 462, 493, 468, 91, 451, 484, 220, 470, 467, 288]


CharacterTextSplitter와 달리 chunk_size=500을 정확히 지켜서 텍스트를 분할한다. 

사용자 지정 청크 길이를 기반으로 문서를 분할하면 기계적으로 분할하기 때문에 문서의 맥락이 끊길수도 있다.   
문장 간의 거리가 이상치에 해당할 정도로 높은 지점에서 문장을 분리하는 SemanticChunker    
문장 간의 거리를 알기 위해 문장마다 임베딩 과정을 거친다.   
기본적으로 문장 간의 거리가 95%를 초과하면 이상 값으로 간주한다.   
실제로는 문장 하나하나가 아니라 N개의 문장을 window로 삼고, 이 window를 점차 우측으로 옮겨가며 그룹 간 거리를 측정한다.   

In [33]:
# 문맥을 파악해 분할하는 SemanticChunker 

import os
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("./data/BOK 이슈노트 제2022-38호 인공지능 언어모형을 이용한 인플레이션 어조지수 개발 및 시사점.pdf")
pages = loader.load()

from langchain_experimental.text_splitter import SemanticChunker
from langchain_google_genai import GoogleGenerativeAIEmbeddings

api_key=os.environ["GEMINI_API_KEY"]

# text_splitter = SemanticChunker(GoogleGenerativeAIEmbeddings(
#     model="models/text-embedding-gecko-001",
#     google_api_key=api_key))
# texts_bok = text_spliter.split_documents(pages)

text_splitter = SemanticChunker(
    GoogleGenerativeAIEmbeddings(
        model="models/embedding-001",
        google_api_key=api_key
    ),
    number_of_chunks=10
)
texts_bok = text_splitter.split_documents(pages)

print("-"*100)
print("[첫번째 청크]")
print(texts_bok[0].page_content)
print("-"*100)
print("[두번째 청크]")
print(texts_bok[1].page_content)

----------------------------------------------------------------------------------------------------
[첫번째 청크]
1
인공지능 언어모형을 이용한 인플레이션 어조지수 개발 및 시사점
뉴스기사, 소셜미디어와 같은 텍스트 데이터는 크게 두 가지 경로를 
통해 경제 분석에 유용한 정보를 제공할 수 있다. 첫째, 지정학적 리스
크, 이상기후 등 이벤트는 수입물가지수와 같은 공식통계에 앞서 뉴스 
속보나 인터넷 커뮤니티에 먼저 반영되며 향후 인플레이션 수준에 대한 
정보를 제공한다.
----------------------------------------------------------------------------------------------------
[두번째 청크]
둘째, 거시경제 여건, 기업의 가격설정 행태 등에 대
한 언론 보도는 기업의 인플레이션 기대를 변화시킴으로써 의사결정에 
영향을 미칠 수 있다. 그러나, 이러한 중요성에도 불구하고, 텍스트 데
이터는 계량화가 쉽지 않아 경제 분석에 활용하는 데 어려움이 크다는 
평가도 있다(로버트 쉴러, 2013년 노벨 경제학상 수상). 최근 정보통신 및 인공지능 기술과 함께 텍스트 분석기법이 빠르게 
발전하면서 경제 분석에 텍스트를 활용하려는 시도가 점차 늘고 있다.


In [34]:
print([len(i.page_content) for i in texts_bok])

[202, 255, 322, 277, 368, 95, 175, 149, 53, 274, 280, 260, 72, 278, 283, 293, 237, 181, 114, 6, 132, 37, 60, 216, 422, 258, 150, 54, 236, 326, 628, 108, 141, 206, 120, 62, 115, 205, 187, 40, 235, 97, 414, 156, 350, 53, 181, 120, 250, 122, 188, 123, 248, 283, 72, 368, 60, 239, 66, 248, 441, 13, 170, 81, 27, 116, 91, 70, 202, 181, 212, 128, 77, 122, 104, 576, 97, 307, 64, 65, 192, 83, 335, 87, 127, 66, 174, 50, 216, 62, 388, 87, 2, 76, 70, 43, 60, 84, 399, 436, 60, 42, 97, 97, 380, 21, 90, 184, 128, 105, 63, 117, 218, 270, 106, 984, 269, 280, 841, 753, 573, 118, 14, 253, 149, 38, 84, 242, 619, 7, 309, 125, 343, 268, 312, 266, 455, 15, 227, 105]


In [20]:
import inspect
from langchain_experimental.text_splitter import SemanticChunker

print(inspect.signature(SemanticChunker.__init__))

(self, embeddings: langchain_core.embeddings.embeddings.Embeddings, buffer_size: int = 1, add_start_index: bool = False, breakpoint_threshold_type: Literal['percentile', 'standard_deviation', 'interquartile', 'gradient'] = 'percentile', breakpoint_threshold_amount: Optional[float] = None, number_of_chunks: Optional[int] = None, sentence_split_regex: str = '(?<=[.?!])\\s+', min_chunk_size: Optional[int] = None)
